In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from utils.StudentLifeDataLoader import StudentLifeDataLoader
from functools import reduce

In [2]:
seed = 24091993

In [3]:

studentlife = StudentLifeDataLoader('studentlife')


In [4]:
stress_data = studentlife.get_stress_data()
social_data = studentlife.get_social_data()
sleep_data = studentlife.get_sleep_data()
conversation_data = studentlife.get_conversation_data()

In [5]:
df_merged = reduce(lambda left,right: pd.merge(left,right,on=['user_id', 'date'],
                                            how='outer'), [stress_data, social_data, sleep_data, conversation_data])

In [6]:
df = df_merged.copy()

In [7]:
# Convert the 'date' column to datetime if it's not already in datetime format
df['date'] = pd.to_datetime(df['date'])

df.set_index('date', inplace=True)
# Group by 'user_id'
grouped = df.groupby('user_id')

In [8]:
# Define a function to filter each group
def filter_group(group):
    # Find the first and last non-null stress_level index
    first_non_null = group['stress_level'].first_valid_index()
    last_non_null = group['stress_level'].last_valid_index()
    
    # If either first_non_null or last_non_null is not found, return an empty DataFrame
    if first_non_null is None or last_non_null is None:
        return pd.DataFrame(columns=group.columns)
    
    # Slice the group based on first and last non-null index
    return group.loc[first_non_null:last_non_null]

def interpolate_column(group, column):
    group[column] = group[column].interpolate(method='nearest')
    return group

In [9]:
# Apply the filtering function to each group and concatenate the results
filtered_df = pd.concat([filter_group(group) for _, group in grouped])

# Reset the index if needed
filtered_df = filtered_df.reset_index()

In [10]:
# interpolate the social level by nearest for each user_id
# Group by 'user_id'
grouped = filtered_df.groupby('user_id')

# Apply the interpolation function to each group and concatenate the results
interpolated_df = grouped.apply(lambda x: interpolate_column(x, 'social_level'))

# Reset the index if needed
interpolated_df = interpolated_df.reset_index(drop=True)

grouped = interpolated_df.groupby('user_id')

# Apply the interpolation function to each group and concatenate the results
interpolated_df = grouped.apply(lambda x: interpolate_column(x, 'sleep_duration'))

# Reset the index if needed
interpolated_df = interpolated_df.reset_index(drop=True)

/tmp/ipykernel_6003/151482971.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  interpolated_df = grouped.apply(lambda x: interpolate_column(x, 'social_level'))
/tmp/ipykernel_6003/151482971.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  interpolated_df = grouped.apply(lambda x: interpolate_column(x, 'sleep_duration'))


In [11]:
interpolated_df.drop_duplicates(['date', 'user_id']).dropna(subset=['stress_level'])

,date,stress_level,user_id,environmental_temperature_mean,environmental_temperature_max,environmental_temperature_min,environmental_humidity_mean,environmental_humidity_max,environmental_humidity_min,environmental_precipitation,...,openness,conscientiousness,flourishing_score,loneliness_score,social_level,sleep_duration,social_voice_sum,social_voice_count,social_voice_mean,social_voice_max
0,2013-03-27,1.0,4,2.0,2.0,2.0,2.0,3.0,1.0,0.0,...,17.0,0.0,17.0,31.0,3.0,6.0,25142.0,41.0,613.219512,3469.0
1,2013-03-28,1.0,4,2.0,2.0,2.0,3.0,4.0,1.0,0.0,...,17.0,0.0,17.0,31.0,3.0,6.0,25256.0,37.0,682.594595,3328.0
2,2013-03-29,2.0,4,2.0,2.0,2.0,3.0,4.0,2.0,0.0,...,17.0,0.0,17.0,31.0,3.0,6.0,28051.0,39.0,719.256410,4280.0
3,2013-03-30,1.0,4,2.0,2.0,2.0,2.0,4.0,0.0,0.0,...,17.0,0.0,17.0,31.0,3.0,7.0,17375.0,33.0,526.515152,2321.0
4,2013-03-31,1.0,4,2.0,2.0,2.0,1.0,2.0,1.0,0.0,...,17.0,0.0,17.0,31.0,3.0,7.0,26301.0,37.0,710.837838,4276.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,2013-05-20,3.0,58,3.0,3.0,3.0,3.0,4.0,2.0,0.0,...,24.0,8.0,41.0,28.0,1.0,6.0,6527.0,18.0,362.611111,836.0
1247,2013-05-21,2.0,58,3.0,3.0,4.0,4.0,4.0,3.0,0.0,...,24.0,8.0,41.0,28.0,1.0,7.0,17384.0,22.0,790.181818,3853.0
1248,2013-05-22,1.0,58,3.0,3.0,3.0,4.0,4.0,2.0,0.0,...,24.0,8.0,41.0,28.0,1.0,9.0,15010.0,18.0,833.888889,6922.0
1249,2013-05-23,3.0,58,3.0,3.0,4.0,4.0,4.0,3.0,0.0,...,24.0,8.0,41.0,28.0,1.0,9.0,32192.0,45.0,715.377778,3660.0


In [12]:
interpolated_df = interpolated_df.dropna(subset=['stress_level'])

interpolated_df['weekday'] = interpolated_df['date'].dt.dayofweek

In [13]:
# stress level from 1 2 3 to 0 1 2
interpolated_df['stress_level'] = interpolated_df['stress_level'].apply(lambda x: 0 if x == 1 else 1 if x == 2 else 2 if x == 3 else 3)

In [14]:
# machine learning classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, StratifiedGroupKFold, StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

np.random.seed(seed)

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.5,
    early_stopping_rounds=20,
    verbose=False,
    loss_function='MultiClass'
)

model = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1)

scaler = StandardScaler()
ambient_columns = [i for i in interpolated_df.columns if i.startswith('ambient_')]
X = interpolated_df.drop(columns=['stress_level', 'date'])
X = scaler.fit_transform(X)
y = interpolated_df['stress_level']


def kfold_cross_validation(model, X, y, n_folds=10):
    kf = KFold(n_splits=n_folds, shuffle=True)
    scores = cross_val_score(model, X, y, cv=kf, scoring='f1_weighted')
    return scores

scores = kfold_cross_validation(model, X, y)
print(scores)
print(scores.mean())


[0.50427297 0.5343487  0.43916141 0.37120454 0.54024915 0.44982297
 0.5149929  0.4614231  0.54100726 0.47017223]
0.48266552220951653


In [15]:
# machine learning classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, StratifiedGroupKFold, StratifiedKFold, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

np.random.seed(seed)

model = CatBoostClassifier(verbose=False, random_seed=63,early_stopping_rounds=20,)

'''
model = CatBoostClassifier(
    iterations=2000,
    random_seed=63,
    learning_rate=0.5,
    early_stopping_rounds=20,
    verbose=False,
    loss_function='MultiClass'
)'''
parameters = {'depth'         : [3, 4,5,6,7,8,9, 10, 16],
                 'learning_rate' : [0.05, 0.1, 0.25, 0.4, 0.5, 0.7],
                  'iterations'    : [200, 500, 1000, 2000],
                 }
#model = XGBClassifier(n_estimators=10, max_depth=2, learning_rate=1)


scaler = MinMaxScaler()
ambient_columns = [i for i in interpolated_df.columns if i.startswith('ambient_')]
X = interpolated_df.drop(columns=['stress_level', 'date'])
X = scaler.fit_transform(X)
y = interpolated_df['stress_level']

# split
Grid_CBC = RandomizedSearchCV(estimator=model, param_distributions = parameters, cv = 5, n_jobs=-1, scoring='f1_weighted')
Grid_CBC.fit(X, y)

print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

def kfold_cross_validation(model, X, y, n_folds=5):
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kf, scoring='f1_weighted')
    return scores

model.set_params(**Grid_CBC.best_params_)

scores = kfold_cross_validation(model, X, y)
print(scores)
print(scores.mean())


KeyboardInterrupt: 

In [ ]:
interpolated_df

,date,stress_level,user_id,ambient_temperature_mean,ambient_temperature_max,ambient_temperature_min,ambient_humidity_mean,ambient_humidity_max,ambient_humidity_min,ambient_precipitation,ambient_cloudcover,extraversion,agreeableness,neuroticism,openness,conscientiousness,social_level,sleep_duration,weekday
0,2013-03-27,1.0,4,0.466667,7.2,-6.1,64.125000,75.0,46.0,0.0,27.791667,1.0,4.0,15.0,17.0,0.0,3.0,6.0,2
1,2013-03-28,1.0,4,3.450000,8.0,0.9,76.333333,95.0,47.0,1.5,84.541667,1.0,4.0,15.0,17.0,0.0,3.0,6.0,3
2,2013-03-29,2.0,4,3.354167,8.6,-1.6,75.833333,95.0,55.0,1.3,27.250000,1.0,4.0,15.0,17.0,0.0,3.0,6.0,4
3,2013-03-30,1.0,4,3.141667,9.0,-2.6,69.750000,95.0,30.0,0.0,19.041667,1.0,4.0,15.0,17.0,0.0,3.0,7.0,5
4,2013-03-31,1.0,4,4.675000,10.9,-1.7,49.125000,65.0,35.0,0.0,35.291667,1.0,4.0,15.0,17.0,0.0,3.0,7.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,2013-05-20,3.0,58,15.695833,22.6,11.0,85.833333,100.0,63.0,1.9,99.916667,-1.0,15.0,13.0,24.0,8.0,1.0,6.0,0
1038,2013-05-21,2.0,58,18.033333,24.4,13.9,87.875000,97.0,67.0,5.5,59.125000,-1.0,15.0,13.0,24.0,8.0,1.0,7.0,1
1039,2013-05-22,1.0,58,14.208333,24.5,8.5,87.708333,99.0,63.0,6.2,96.000000,-1.0,15.0,13.0,24.0,8.0,1.0,9.0,2
1040,2013-05-23,3.0,58,18.450000,24.7,13.7,88.083333,99.0,68.0,1.9,93.666667,-1.0,15.0,13.0,24.0,8.0,1.0,9.0,3
